In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
dfs = []

for year in range(2015, 2026):
    dummy = pd.read_csv(f"nyc_2015-2025/nyc_{year}.csv")

    dfs.append(dummy)

In [3]:
df = pd.DataFrame()
df = pd.concat(dfs, ignore_index=True)

In [4]:
# Remove useless columns
df.drop(columns=['AA1', 'AA2', 'AA3', 'AB1', 'AD1', 'AE1',
       'AH1', 'AH2', 'AH3', 'AH4', 'AH5', 'AH6', 'AI1', 'AI2', 'AI3', 'AI4',
       'AI5', 'AI6', 'AJ1', 'AK1', 'AM1', 'AN1', 'AT1', 'AT2', 'AT3', 'AT4',
       'AT5', 'AT6', 'AT7', 'AT8', 'AU1', 'AU2', 'AW1', 'AW2', 'AW3', 'AX1',
       'AX2', 'AX3', 'AX4', 'GA1', 'GA2', 'GA3', 'GD1', 'GD2', 'GD3', 'GE1',
       'GF1', 'KA1', 'KA2', 'KB1', 'KB2', 'KB3', 'KC1', 'KC2', 'KD1', 'KD2',
       'KE1', 'KG1', 'KG2', 'MA1', 'MD1', 'MF1', 'MG1', 'MH1', 'MK1', 'MW1',
       'OC1', 'OD1', 'OE1', 'OE2', 'OE3', 'RH1', 'RH2', 'RH3', 'WA1', 'REM',
       'EQD', 'GJ1', 'AL1', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'STATION',
                   'REPORT_TYPE', 'QUALITY_CONTROL'], inplace=True)

In [5]:
# Rename columns to readable names
df.rename(columns={'TMP' : 'temp',
                     'CIG' : 'cloud_ceiling_height', 
                     'VIS' : 'visibility', 
                     'DEW' : 'dew_point_pressure',
                     'SLP' : 'sea_level_pressure',
                     'WND' : 'wind',
                     'DATE' : 'datetime',
                     'CALL_SIGN': 'call_sign'},
                     inplace=True)

In [6]:
# Convert the 'date' feature from a string object to a true datetime object, then set it as the index
df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
df = df.set_index('datetime')

df["date"] = df.index.normalize() # midnight stamp

df = df[['date', 'call_sign', 'temp', 'cloud_ceiling_height', 'visibility', 'dew_point_pressure', 'sea_level_pressure', 'wind']]

In [15]:
"""
### Analyzing invalid temperature measruements by time

# 1. Filter out the “invalid” rows where temp starts with '+9999'
invalids = df[df['temp'].str.startswith('+9999')]

# 2. Count how many invalids occur in each hour (0–23)
freq_by_hour = (
    invalids
    .groupby(invalids.index.hour)
    .size()
    .reindex(range(24), fill_value=0)
)

# 3. Plot a bar chart
plt.figure(figsize=(10, 5))
freq_by_hour.plot(kind='bar')
plt.xlabel('Hour of Day')
plt.ylabel('Count of Invalid Measurements')
plt.title('Invalid Temperature Measurements by Hour')
plt.xticks(range(24), [f"{h}:00" for h in range(24)], rotation=45)
plt.tight_layout()
plt.show()
"""

'\n# 1. Filter out the “invalid” rows where temp starts with \'+9999\'\ninvalids = df[df[\'temp\'].str.startswith(\'+9999\')]\n\n# 2. Count how many invalids occur in each hour (0–23)\nfreq_by_hour = (\n    invalids\n    .groupby(invalids.index.hour)\n    .size()\n    .reindex(range(24), fill_value=0)\n)\n\n# 3. Plot a bar chart\nplt.figure(figsize=(10, 5))\nfreq_by_hour.plot(kind=\'bar\')\nplt.xlabel(\'Hour of Day\')\nplt.ylabel(\'Count of Invalid Measurements\')\nplt.title(\'Invalid Temperature Measurements by Hour\')\nplt.xticks(range(24), [f"{h}:00" for h in range(24)], rotation=45)\nplt.tight_layout()\nplt.show()\n'

In [16]:
'''
invalid_readings = df[df['temp'] == '+9999,9']
invalid_readings.iloc[5:10,:]
'''

"\ninvalid_readings = df[df['temp'] == '+9999,9']\ninvalid_readings.iloc[5:10,:]\n"

In [7]:
# Dropping any invalid rows
df = df[df['temp'] != '+9999,9'].reset_index(drop=True)

In [8]:
# Converting temperature to the true temperature int value reading
df['temp'] = (
    df['temp']
      .str.replace(r',.*$', '', regex=True)   # drop comma + quality code
      .astype(int)                            # "+0234" → 234
      .div(10)                                # 234 → 23.4°C
)

In [ ]:
# Calculating highest temperature so far


In [16]:
### Removing invalid data points from the rows and setting them to NaN

# Removing invalid df from the wind column
parts = df['wind'].str.split(',', expand=True)
df.drop(columns=['wind'],inplace=True)

In [17]:
parts[0].replace('999', np.nan, inplace=True)
parts[2].replace('9', np.nan, inplace=True)
parts[3].replace('9999', np.nan,inplace=True)

parts.drop(columns=[1,4],inplace=True)

In [18]:
parts.head()

# Column 1: Direction angle in degrees
# Column 2: Wind observation type code
# Column 3: Wind observation speed rate (in 0.1 m/s)

,0,2,3
0,260,N,0021
1,240,N,0015
2,NaN,V,0021
3,NaN,NaN,NaN
4,NaN,V,0021


In [19]:
df['wind_dir'] = pd.to_numeric(parts[0], errors='coerce')
df['wind_type_code'] = parts[2]
df['wind_spd'] = pd.to_numeric(parts[3], errors='coerce')

In [20]:
df.head()

,DATE,CALL_SIGN,temp,cloud_ceiling_height,visibility,dew_point_pressure,sea_level_pressure,wind_dir,wind_type_code,wind_spd
0,2015-01-01T00:51:00,KNYC,"-0011,5","22000,5,9,N","016093,5,N,5","-0139,5","10235,5",260.0,N,21.0
1,2015-01-01T01:51:00,KNYC,"-0011,5","22000,5,9,N","016093,5,N,5","-0144,5","10230,5",240.0,N,15.0
2,2015-01-01T02:51:00,KNYC,"-0011,5","22000,5,9,N","016093,5,N,5","-0133,5","10223,5",NaN,V,21.0
3,2015-01-01T03:51:00,KNYC,"-0017,5","22000,5,9,N","016093,5,N,5","-0128,5","10220,5",NaN,NaN,NaN
4,2015-01-01T04:51:00,KNYC,"-0022,5","22000,5,9,N","016093,5,N,5","-0128,5","10218,5",NaN,V,21.0


In [30]:
# 1) Vector components (u, v) at each hour  ────────────────────────────
θ               = np.deg2rad(df["wind_dir"])        # deg → rad
df["u"]         = df["wind_spd"] * np.sin(θ)        # +u = eastward
df["v"]         = df["wind_spd"] * np.cos(θ)        # +v = northward
df["dir_sin"]   = np.sin(θ)                         # keep raw sin / cos too
df["dir_cos"]   = np.cos(θ)

In [31]:
# 2) Gustiness *so far today*  (prefix-day)
df["gust_flag"] = (df["wind_spd"] > 8).astype(int)  # tweak threshold as needed

# expanding count & index (max-min) up through t_pred
df["gust_cnt_to_now"] = (
    df.groupby("date")["gust_flag"]
      .expanding()
      .sum()
      .reset_index(level=0, drop=True)
)

df["gust_idx_to_now"] = (
    df.groupby("date")["wind_spd"]
      .expanding()
      .apply(lambda s: s.max() - s.min(), raw=True)
      .reset_index(level=0, drop=True)
)

In [32]:
# 3) Directional persistence: 24-h circular variance (uses *past* 24 h only)
θ = np.deg2rad(df["wind_dir"])
sin24 = np.sin(θ).rolling(24, min_periods=3).mean()   # need ≥3 pts for stability
cos24 = np.cos(θ).rolling(24, min_periods=3).mean()
df["dir_var24"] = 1.0 - np.sqrt(sin24**2 + cos24**2)

In [ ]:
# 4) Sector fractions *so far today* (N, E, S, W)  ─────────────────────
# --- 1. Bin wind directions into sectors -----------------------------
bins   = [-1, 45, 135, 225, 315, 360]         # edges in degrees
labels = ["N", "E", "S", "W", "N"]            # wrap 0–45° and 315–360° to "N"

sector = pd.cut(
    df["wind_dir"],
    bins=bins,
    labels=labels,
    ordered=False            # <-- allows duplicate "N" labels
)

# --- 2. One-hot encode each sector (hourly 0/1 flags) -----------------
sec_dum = pd.get_dummies(sector, prefix="sec")   # e.g. sec_N, sec_E, ...

# attach the dummies to the main DataFrame
df = df.join(sec_dum)

# --- 3. Running (expanding) fraction of hours in each sector ---------
# create a "date" column: today's midnight stamp
df["date"] = df.index.normalize()

# for every sector dummy, compute the expanding mean within each day
for col in sec_dum.columns:
    df[f"{col}_frac_to_now"] = (
        df.groupby("date")[col]            # group by calendar day
          .expanding()                     # cumulative window
          .mean()                          # running fraction
          .reset_index(level=0, drop=True) # align back to original index
    )

# Now at the current timestamp (e.g. 12:00) each *_frac_to_now column
# holds the fraction of the day’s hours so far that came from that sector.

In [ ]:
# 5)Lagged wind snapshot (1 h, 3 h, 6 h before t_pred) ────────────────
for lag in [1, 3, 6]:
    df[f"u_lag{lag}h"]   = df["u"].shift(lag)
    df[f"v_lag{lag}h"]   = df["v"].shift(lag)
    df[f"spd_lag{lag}h"] = df["wind_spd"].shift(lag)
# At prediction time row (t_pred) these hold the desired lags.

In [ ]:
# 6) Calm-hour fraction so far today  ───────────────────────────────────
df["calm"] = (df["wind_spd"] < 1).astype(int)

df["calm_frac_to_now"] = (
    df.groupby("date")["calm"]
      .expanding()
      .mean()
      .reset_index(level=0, drop=True)
)

In [ ]:
# 7) Wind-type categorical fractions so far today  ──────────────────────
wt_dum = pd.get_dummies(df["wind_type"].fillna("M"), prefix="wt")
df[wt_dum.columns] = wt_dum

for col in wt_dum.columns:
    df[f"{col}_frac_to_now"] = (
        df.groupby("date")[col]
          .expanding()
          .mean()
          .reset_index(level=0, drop=True)
    )

In [ ]:
# 8) Seasonal interaction (works fine mid-day)  ─────────────────────────
df["doy"] = df.index.dayofyear
ω         = 2 * np.pi * df["doy"] / 365.25
df["u_doysin"] = df["u"] * np.sin(ω)
df["v_doycos"] = df["v"] * np.cos(ω)

In [22]:
df.head()

,date,call_sign,temp,cloud_ceiling_height,visibility,dew_point_pressure,sea_level_pressure,wind
0,2015-01-01 00:00:00+00:00,KNYC,-1.1,"22000,5,9,N","016093,5,N,5","-0139,5","10235,5","260,5,N,0021,5"
1,2015-01-01 00:00:00+00:00,KNYC,-1.1,"22000,5,9,N","016093,5,N,5","-0144,5","10230,5","240,5,N,0015,5"
2,2015-01-01 00:00:00+00:00,KNYC,-1.1,"22000,5,9,N","016093,5,N,5","-0133,5","10223,5","999,9,V,0021,5"
3,2015-01-01 00:00:00+00:00,KNYC,-1.7,"22000,5,9,N","016093,5,N,5","-0128,5","10220,5","999,9,9,9999,9"
4,2015-01-01 00:00:00+00:00,KNYC,-2.2,"22000,5,9,N","016093,5,N,5","-0128,5","10218,5","999,9,V,0021,5"
